# Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

In [1]:
import pandas as pd
import numpy as np
import os

def answer_one():
    file_name = 'Energy Indicators.xls'
    energy = pd.read_excel(file_name, skiprows=17)
    energy.drop(columns=['Unnamed: 0', 'Unnamed: 1'], inplace=True)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    
    for col in ['Energy Supply', 'Energy Supply per Capita']:
        energy[col].where(cond=(energy[col].str.isnumeric()), other=np.NaN, inplace=True)
        energy[col].astype(float)
    
    energy['Energy Supply'] *= 1000000
    energy['Country'] = energy['Country'].str.replace("\d+", "")
    energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")
    
    dict = {
        "Republic of Korea" : 'South Korea',
        "United States of America" : 'United States',
        "United Kingdom of Great Britain and Northern Ireland" : 'United Kingdom',
        "China, Hong Kong Special Administrative Region" : 'Hong Kong',
    }
    
    energy['Country'] = energy['Country'].replace(dict, regex=True) 
    
    file_name = 'world_bank.csv'
    GDP = pd.read_csv(file_name, skiprows=4)
    
    dict = {
        "Korea, Rep.": 'South Korea',
        "Iran, Islamic Rep." : 'Iran',
        "Hong Kong SAR, China" : 'Hong Kong',
    }
    
    GDP['Country Name'] = GDP['Country Name'].replace(dict, regex=True) 
    
    file_name = 'ScimEn.xlsx'
    ScimEn = pd.read_excel(file_name)
    
    GDP.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 65'], axis=1, inplace=True)
    
    for col in GDP.columns:
        if col.isnumeric() and (int(col) < 2006 or int(col) > 2015):
            GDP.drop(col, axis=1, inplace=True)
            
    merge_result = pd.merge(energy, GDP, left_on='Country', right_on='Country Name').drop('Country Name', axis=1)
    merge_result = pd.merge(ScimEn.head(15), merge_result, how='inner').drop('Region', axis=1)
    merge_result.set_index('Country', inplace=True)
    
    return merge_result

answer_one()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,273437,272374,2336764,1615239,8.55,245,127191000000,93,19.754910,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,175891,172431,2230544,724472,12.68,363,90838000000,286,11.570980,1.381461e+13,1.445186e+13,1.471284e+13,1.444893e+13,1.499205e+13,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.822470e+13
India,3,55082,53775,463165,162944,8.41,181,33195000000,26,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,50523,50065,488062,119930,9.66,193,18984000000,149,10.232820,4.530377e+12,4.515265e+12,5.037908e+12,5.231383e+12,5.700098e+12,6.157460e+12,6.203213e+12,5.155717e+12,4.850414e+12,4.389476e+12
United Kingdom,5,43389,42284,615670,111290,14.19,226,7920000000,124,10.600470,2.713750e+12,3.100882e+12,2.922667e+12,2.410910e+12,2.475244e+12,2.659310e+12,2.704888e+12,2.786023e+12,3.063803e+12,2.928591e+12
Germany,6,38739,38013,433148,95145,11.18,196,13261000000,165,17.901530,2.992197e+12,3.421229e+12,3.730028e+12,3.397791e+12,3.396354e+12,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.356236e+12
Russian Federation,7,36735,36560,115938,54993,3.16,90,30709000000,214,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,33472,32863,568080,100953,16.97,227,10431000000,296,61.945430,1.315415e+12,1.464977e+12,1.549131e+12,1.371153e+12,1.613464e+12,1.788648e+12,1.828689e+12,1.847209e+12,1.803533e+12,1.556130e+12
Italy,9,27983,26940,352993,87828,12.61,166,6530000000,109,33.667230,1.947920e+12,2.210293e+12,2.398857e+12,2.191242e+12,2.134018e+12,2.291991e+12,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12


## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [2]:
def answer_two():
    Top15 = answer_one()
    
    cols = []
    for col in Top15.columns:
        if col.isnumeric():
            cols.append(col)
    avgGDP = pd.Series(Top15[cols].mean(axis=1)).sort_values(ascending=False)
    return avgGDP

answer_two()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


Country
United States         1.566966e+13
China                 6.927702e+12
Japan                 5.177131e+12
Germany               3.518225e+12
United Kingdom        2.776607e+12
France                2.687371e+12
Italy                 2.139775e+12
Brazil                1.988885e+12
Russian Federation    1.666746e+12
Canada                1.613835e+12
India                 1.602352e+12
Spain                 1.398587e+12
South Korea           1.221372e+12
Australia             1.206526e+12
Iran                  4.389286e+11
dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [3]:
def answer_three():
    avgGDP = answer_two()
    Top15 = answer_one()
   
    index = avgGDP.index[5]
    return Top15['2015'].loc[index] - Top15['2006'].loc[index]
answer_three()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


119614244263.37988

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [4]:
def answer_four():
    Top15 = answer_one()
    Top15['Сitations_attitude'] = Top15['Self-citations'] / Top15['Citations']
    return (Top15[Top15['Сitations_attitude'] == Top15['Сitations_attitude'].max()].index[0], Top15['Сitations_attitude'].max())

answer_four()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


('China', 0.6912289816173135)

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [5]:
def answer_five():
    Top15 = answer_one()
    Top15['Energy_supply_estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    return Top15['Energy_supply_estimate'].sort_values(ascending=False).index[2]

answer_five()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


'United States'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [6]:
def answer_six():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable_documents_per_person'] = Top15['Citable documents'] / Top15['Population']
    
    return Top15[['Energy Supply per Capita', 'Citable_documents_per_person']].astype(float).corr().iloc[0, 1]
answer_six()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


0.7434709127726777

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [7]:
def answer_seven():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = Top15.astype(float)
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    grouped = pd.DataFrame()
    grouped['size'] = Top15['Population'].groupby(by=ContinentDict,axis=0).count()
    grouped['sum'] = Top15['Population'].groupby(by=ContinentDict,axis=0).sum()
    grouped['mean'] = Top15['Population'].groupby(by=ContinentDict,axis=0).mean()
    grouped['std'] = Top15['Population'].groupby(by=ContinentDict,axis=0).std()
    grouped.index.names = ['Continent']
    
    return grouped

answer_seven()

<ipython-input-1-a550c51afab7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace("\d+", "")
<ipython-input-1-a550c51afab7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  energy['Country'] = energy['Country'].str.replace(r' \([^)]*\)',"")


,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN
